In [14]:
import numpy as np
import pandas as pd

In [15]:
# k = 1000
# split_pair_path = "data/split_dev/split_pair_dev_k_{0}_all.txt".format(k)

k = 100
split_pair_path = "data/split_dev/split_pair_dev_k_{0}.txt".format(k)


#k=100
#split_pair_path = "data/first_filter_2epoch/new_split_pair_dev_2epoch_k_{0}.txt".format(k)


split_data = pd.read_csv(split_pair_path, sep='\t')


In [16]:
len(split_data)

3282700

In [17]:
%%time
total_len = len(split_data)
unique_len = int(total_len/k)
original_shape = (unique_len, k)
true_sentence = np.array(split_data.true_sentence).reshape(original_shape)
candidate_sentence = np.array(split_data.candidate_sentence).reshape(original_shape)
pair_result = np.array(split_data.pair_result).reshape(original_shape)
true_label = np.array(split_data.true_label).reshape(original_shape)
candidate_label = np.array(split_data.candidate_label).reshape(original_shape)

CPU times: user 498 ms, sys: 38.3 ms, total: 536 ms
Wall time: 401 ms


In [18]:
# from probability file, get top k candidates
probability_file_path = "data/split_dev/split_pair_dev_k_{0}_prob_3_sep.csv".format(k)
#probability_file_path = "data/first_filter_2epoch/test_results_k_{0}_of_k_1000.csv".format(k)
#probability_file_path = "data/first_filter_2epoch/test_results_k_{0}_split_all.csv".format(k)


probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]
probability_mat = np.array(probability_file).reshape(original_shape)

In [19]:
%%time
sorted_pair_result = np.zeros((original_shape)).astype('int')
sorted_candidate_sentence = np.zeros((original_shape), dtype=object)
sorted_candidate_label = np.zeros((original_shape)).astype('int')
for i in range(unique_len):
    sorted_index = probability_mat[i].argsort()[::-1]
    sorted_pair_result[i] = pair_result[i, sorted_index]
    sorted_candidate_sentence[i] = candidate_sentence[i, sorted_index]
    sorted_candidate_label[i] = candidate_label[i, sorted_index]

CPU times: user 1.14 s, sys: 47.3 ms, total: 1.19 s
Wall time: 1.19 s


In [24]:
top = 50
np.mean(np.max(sorted_pair_result[:,:top], axis=1))
#np.mean(sorted_pair_result[:,:top].ravel())

0.4861546897371066

In [11]:
def getOutput(mat, index, top):
    return mat[index, :top].ravel()

In [12]:
positive_index = (np.max(sorted_pair_result[:,:top], axis=1) == 1)
output_true_sentence = getOutput(true_sentence, positive_index, top)
output_candidate_sentence =  getOutput(sorted_candidate_sentence, positive_index, top)
output_pair_result = getOutput(sorted_pair_result, positive_index, top)
output_true_label = getOutput(true_label, positive_index, top)
output_candidate_label = getOutput(sorted_candidate_label, positive_index, top)

In [13]:
output_file_path = "data/first_filter_2epoch/new_split_pair_dev_2epoch_k_{0}.txt".format(top)
pair_df = pd.DataFrame({'true_sentence':output_true_sentence,                               
                         'candidate_sentence':output_candidate_sentence,                     
                         'pair_result':output_pair_result,                         
                         'true_label':output_true_label,                                     
                         'candidate_label':output_candidate_label})                          
pair_df.to_csv(output_file_path, sep='\t', index=False) 